<a href="https://colab.research.google.com/github/Henil21/GAN_AI/blob/main/GAN_Clouds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# data_train='/content/drive/MyDrive/cloud/'
import tensorflow as tf
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
import time
from tensorflow.keras.utils import load_img, array_to_img
import cv2 as cv
from IPython import display
import matplotlib.image as mpimg
from tqdm.notebook import tqdm
from tensorflow import keras

In [ ]:
data_train='/content/drive/MyDrive/cloud/clouds'
import os
images=[]
for image_name in  os.listdir(data_train):
  image_path=os.path.join(data_train,image_name)
  images.append(image_path)
  images=images[:400]

In [ ]:
# def visualize_Images(images_data):

#     import random
#     visul=random.sample(images_data,14)
#     plt.figure(figsize=(20,20))
#     temp_img=visul[:14]

#     index=1

#     for imag in temp_img:
#       # image = image / 255.0  # Rescale image values to range 0-1
#       plt.subplot(7,7,index)
#       # print(imag)
#       image=load_img(imag)
#       image=np.array(image)
#       plt.imshow(image)
#       plt.axis("off")
#       index+=1

In [ ]:
# visualize_Images(images)

In [ ]:
train_images=[np.array(load_img(path)) for path in tqdm(images)]
train_images=np.array(train_images)

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
train_images=train_images.reshape(train_images.shape[0],512,512,3).astype('float32')

In [ ]:
train_images=(train_images-127.5)/127.5
# train_images[0]

In [ ]:
LATEN_DIM=100
Weight_init=keras.initializers.RandomNormal(mean=0.0,stddev=0.02)
CHANNELS=3 # 1 if we are using gray-scale

In [ ]:
model=tf.keras.Sequential(name="generator")
model.add(layers.Dense(16*16*512, input_dim=(LATEN_DIM))) # generating random noise
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# converting 1D to 3D we have 3 channels
model.add(layers.Reshape((16, 16, 512)))

#  upstamp to 32x32
model.add(layers.Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=Weight_init))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# upstamp to 64x64
model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=Weight_init))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# upstamp to 128x128
model.add(layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=Weight_init))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# upstamp to 256x256
model.add(layers.Conv2DTranspose(32, (4, 4), strides=(2, 2), padding='same', kernel_initializer=Weight_init))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# upstamp to 512x512
model.add(layers.Conv2DTranspose(16, (4, 4), strides=(2, 2), padding='same', kernel_initializer=Weight_init))
model.add(layers.BatchNormalization())
model.add(layers.ReLU())

# output layer: 512x512 with 3 channels
model.add(layers.Conv2D(CHANNELS, (4, 4), padding='same', activation='tanh'))

generator = model

generator.summary()

Model: "generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 131072)            13238272  
                                                                 
 batch_normalization (BatchN  (None, 131072)           524288    
 ormalization)                                                   
                                                                 
 re_lu (ReLU)                (None, 131072)            0         
                                                                 
 reshape (Reshape)           (None, 16, 16, 512)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 32, 32, 256)      2097408   
 nspose)                                                         
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 256)      10

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


                                                                 
Total params: 16,559,283
Trainable params: 16,296,147
Non-trainable params: 263,136
_________________________________________________________________


In [ ]:
model=tf.keras.Sequential(name='discriminator')
input_shape=(512,512,3)
alpha=0.2

model.add(layers.Conv2D(64,(4,4),strides=(2,2),padding='same',input_shape=input_shape))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha=alpha))

model.add(layers.Conv2D(128,(4,4),strides=(2,2),padding='same',input_shape=input_shape))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha=alpha))

model.add(layers.Conv2D(128,(4,4),strides=(2,2),padding='same',input_shape=input_shape))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(alpha=alpha))

model.add(layers.Flatten())
model.add(layers.Dropout(0.3))

# output class
model.add(layers.Dense(1,activation='sigmoid'))
discriminator=model
discriminator.summary()

Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      3136      
                                                                 
 batch_normalization_6 (Batc  (None, 256, 256, 64)     256       
 hNormalization)                                                 
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 256, 256, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 128)     131200    
                                                                 
 batch_normalization_7 (Batc  (None, 128, 128, 128)    512       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 128, 128, 128)   

In [ ]:
class DCGAN(keras.Model):
    def __init__(self,generator,discriminator,latent_dim):
        super(DCGAN,self).__init__()
        self.generator=generator
        self.discriminator=discriminator
        self.latent_dim=latent_dim

        self.g_loss_metric = keras.metrics.Mean(name='g_loss')
        self.d_loss_metric = keras.metrics.Mean(name='d_loss')


    @property
    def metrics(self):
      return[self.g_loss_metric,self.d_loss_metric]
    def compile(self,g_optimizer,d_optimizer ,loss_fn):
      super(DCGAN,self).compile()
      self.g_optimizer=g_optimizer
      self.d_optimizer=d_optimizer
      self.loss_fn=loss_fn
    def train_step(self,real_images):
      # get batch size
      batch_size=tf.shape(real_images)[0]
      # generate random noise
      random_noise = tf.random.normal(shape=(batch_size, self.latent_dim))

      # train discriminator with range real(1) and fake(0) images
      with tf.GradientTape() as tape:
            # compute loss on real images
            pred_real = self.discriminator(real_images, training=True)
            # generate real image labels
            real_labels = tf.ones((batch_size, 1))
            # label smoothing
            real_labels += 0.05 * tf.random.uniform(tf.shape(real_labels))
            d_loss_real = self.loss_fn(real_labels, pred_real)

            # compute loss on fake images
            fake_images = self.generator(random_noise)
            pred_fake = self.discriminator(fake_images, training=True)
            # generate fake labels
            fake_labels = tf.zeros((batch_size, 1))
            d_loss_fake = self.loss_fn(fake_labels, pred_fake)

            # total discriminator loss
            d_loss = (d_loss_real + d_loss_fake) / 2

        # compute discriminator gradients
      gradients = tape.gradient(d_loss, self.discriminator.trainable_variables)
        # update the gradients
      self.d_optimizer.apply_gradients(zip(gradients, self.discriminator.trainable_variables))


        # train the generator model
      labels = tf.ones((batch_size, 1))
        # generator want discriminator to think that fake images are real
      with tf.GradientTape() as tape:
            # generate fake images from generator
            fake_images = self.generator(random_noise, training=True)
            # classify images as real or fake
            pred_fake = self.discriminator(fake_images, training=True)
            # compute loss
            g_loss = self.loss_fn(labels, pred_fake)

        # compute gradients
      gradients = tape.gradient(g_loss, self.generator.trainable_variables)
        # update the gradients
      self.g_optimizer.apply_gradients(zip(gradients, self.generator.trainable_variables))

        # update states for both models
      self.d_loss_metric.update_state(d_loss)
      self.g_loss_metric.update_state(g_loss)

      return {'d_loss': self.d_loss_metric.result(), 'g_loss': self.g_loss_metric.result()}

In [ ]:
class DCGANMonitor(keras.callbacks.Callback):
    def __init__(self, num_imgs=4, latent_dim=100):
        self.num_imgs = num_imgs
        self.latent_dim = latent_dim
        # create random noise for generating images
        self.noise = tf.random.normal([9, latent_dim])

    def on_epoch_end(self, epoch, logs=None):
        # generate the image from noise
        g_img = self.model.generator(tf.random.normal([4, self.latent_dim]))
        # denormalize the image
        g_img = (g_img * 127.5) + 127.5
        g_img.numpy()

        fig = plt.figure(figsize=(8, 8))
        for i in range(self.num_imgs):
            plt.subplot(2, 2, i+1)
            img = array_to_img(g_img[i])
            plt.imshow(img)
            plt.axis('off')
        # plt.savefig('epoch_{:03d}.png'.format(epoch))
        plt.show()

    def on_train_end(self, logs=None):
        self.model.generator.save('generator.h5')

In [ ]:
dcgan=DCGAN(generator,discriminator,LATEN_DIM)
# checkpoint_callback = ModelCheckpoint('/content/drive/MyDrive/Checkpoints{epoch:03d}.h5', save_freq=10)
# Define the checkpoint callback
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    '/content/drive/MyDrive/Checkpoints/Checkpoints{epoch:03d}.h5',
    save_weights_only=True,
    period=10  # Save weights every 10 epochs
)

In [ ]:
D_LR=0.001
G_LR=0.005
dcgan.compile(g_optimizer=Adam(learning_rate=G_LR,beta_1=0.5),d_optimizer=Adam(learning_rate=D_LR,beta_1=0.5),loss_fn=BinaryCrossentropy())

In [ ]:
N_epoch=130
latest_checkpoint = tf.train.latest_checkpoint("/content/drive/MyDrive/Checkpoints/Checkpoints")
if latest_checkpoint:
    dcgan.load_weights(latest_checkpoint)
dcgan.fit(train_images,epochs=N_epoch,callbacks=[DCGANMonitor(),checkpoint_callback])